# Training on 80% data and then subsequent cross-validation

In [1]:
import warnings
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LinearRegression
import pickle
from sklearn.metrics import accuracy_score


def warn(*args, **kwargs):
    pass


warnings.warn = warn


class Regressor():

    def funcreg(self,reg, X, y, filename):

        X_train, X_test, y_train, y_test = train_test_split( X, y, shuffle=True, test_size=0.20, random_state=42)
        reg.fit(X_train, y_train)
        y_predicted_proba = reg.predict(X_test)
        weig = 0.5 
        #len(y[y == 1])/len(y[y == 0])
        y_predicted = np.where(y_predicted_proba >= weig, 1, 0)
        print("Confusion_matrix for initial training-test set")
        print(confusion_matrix(y_test, y_predicted))
        print("Classification_report for initial training-test set")
        print(classification_report(y_test, y_predicted))
        #print("Training score for initial training-test set")
        #print(reg.score(X_train, y_train))
        #print("Testing score for initial training-test set")
        #print(reg.score(X_test, y_test))
        accuracies = cross_val_score(estimator=reg, X=X, y=y, cv=10)
        print("Cross-validation-accuracies")
        print(accuracies)
        print("Cross-validation-accuracies Mean:")
        print(accuracies.mean())
        print("Cross-validation-accuracies Standard Devaition:")
        print(accuracies.std())
        pickle.dump(reg, open(filename, 'wb'))
    
    def linear_reg(self, X,y):
        np_x_train, np_x_test, np_y_train, np_y_test = train_test_split( X, y, shuffle=True, test_size=0.20, random_state=42)
        np_x_train=np.array(np_x_train)
        np_y_train=np.array(np_y_train)
        np_x_test=np.array(np_x_test)
        np_y_test=np.array(np_y_test)        
        Y_pred = np.empty((np.shape(np_y_test)[0], np.shape(np_y_test)[1]))
        bias = 1
        for i in range(9):
            y = np_y_train[:, i]
            W = np.linalg.inv(np_x_train.T @ np_x_train) @ np_x_train.T @ y
            W = [weight + bias for weight in W]
            y_pred = np_x_test @ W
            Y_pred[:, i] = y_pred

        Y_pred = (Y_pred == Y_pred.max(axis=1)[:, None]).astype(int)

        total_acc = np.empty(9)
        for i in range(9):
            total_acc[i] = self.get_accuracy_score(np_y_test[:, i],
                                                    Y_pred[:, i], normalized=False)

        acc = np.sum(total_acc) / (np.shape(np_y_test)[0] * 9)
        print("Accuracy LR: {0}".format(acc))

    def get_accuracy_score(self,y_true, y_pred, normalized=True):
        pred_accu = accuracy_score(y_true, y_pred, normalize=normalized)
        return pred_accu

if __name__ == '__main__':
    obj = Regressor()
    data_multiple = pd.read_csv('tictac_multi.txt', sep=" ", header=None)
    col_X = [0, 1, 2, 3, 4, 5, 6, 7, 8]
    col_y = [9, 10, 11, 12, 13, 14, 15, 16, 17]
    X_multiple = data_multiple[col_X]
    y_multiple=data_multiple[col_y]

    print("For intermediate boards optimal play(multi-label) dataset:")
    print("KNN Regressor:")
    for i in col_y:
        filename = 'Model_param_col_'+str(i-9)+'.pkl'
        print("For column no: "+str(i-9))
        reg_knn = KNeighborsRegressor(n_neighbors=9,weights='distance',algorithm='kd_tree',p=2,leaf_size=9,n_jobs=-1)
        obj.funcreg(reg_knn, X_multiple, data_multiple[i],filename)

    print("Linear Regressor:")
    obj.linear_reg(X_multiple,y_multiple)

    print("MLP Regressor:")
    for i in col_y:
        filename = 'Model_param_col_'+str(i-9)+'.pkl'
        print("For column no: "+str(i-9))
        reg_knn = MLPRegressor(random_state=42, max_iter=1000,early_stopping=True,solver='lbfgs', activation='tanh')
        obj.funcreg(reg_knn, X_multiple, data_multiple[i],filename)


For intermediate boards optimal play(multi-label) dataset:
KNN Regressor:
For column no: 0
Confusion_matrix for initial training-test set
[[971  17]
 [ 32 291]]
Classification_report for initial training-test set
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       988
           1       0.94      0.90      0.92       323

    accuracy                           0.96      1311
   macro avg       0.96      0.94      0.95      1311
weighted avg       0.96      0.96      0.96      1311

Cross-validation-accuracies
[0.74449855 0.858954   0.83401277 0.8765574  0.87361868 0.92266674
 0.87459051 0.92029613 0.90131005 0.9193329 ]
Cross-validation-accuracies Mean:
0.872583772326226
Cross-validation-accuracies Standard Devaition:
0.05088777617738081
For column no: 1
Confusion_matrix for initial training-test set
[[1075   11]
 [  33  192]]
Classification_report for initial training-test set
              precision    recall  f1-score   support

 

Cross-validation-accuracies
[0.6797569  0.71525777 0.7505676  0.74358179 0.76443807 0.76553469
 0.78658301 0.81157114 0.78058541 0.81042357]
Cross-validation-accuracies Mean:
0.7608299947595466
Cross-validation-accuracies Standard Devaition:
0.038955520029787166
For column no: 3
Confusion_matrix for initial training-test set
[[1038   23]
 [  33  217]]
Classification_report for initial training-test set
              precision    recall  f1-score   support

           0       0.97      0.98      0.97      1061
           1       0.90      0.87      0.89       250

    accuracy                           0.96      1311
   macro avg       0.94      0.92      0.93      1311
weighted avg       0.96      0.96      0.96      1311

Cross-validation-accuracies
[0.52140962 0.62991761 0.61364509 0.69833534 0.69914592 0.68141782
 0.65021661 0.70298788 0.72865016 0.69718065]
Cross-validation-accuracies Mean:
0.6622906706572562
Cross-validation-accuracies Standard Devaition:
0.05805875683846792
For c

# Training on only 10% data and then subsequent cross-validation 

In [3]:
import warnings
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LinearRegression
import pickle
from sklearn.metrics import accuracy_score


def warn(*args, **kwargs):
    pass


warnings.warn = warn


class Regressor():

    def funcreg(self,reg, X, y, filename):

        X_train, X_test, y_train, y_test = train_test_split( X, y, shuffle=True, test_size=0.90, random_state=42)
        reg.fit(X_train, y_train)
        y_predicted_proba = reg.predict(X_test)
        weig = 0.5 
        #len(y[y == 1])/len(y[y == 0])
        y_predicted = np.where(y_predicted_proba >= weig, 1, 0)
        print("Confusion_matrix for initial training-test set")
        print(confusion_matrix(y_test, y_predicted))
        print("Classification_report for initial training-test set")
        print(classification_report(y_test, y_predicted))
        #print("Training score for initial training-test set")
        #print(reg.score(X_train, y_train))
        #print("Testing score for initial training-test set")
        #print(reg.score(X_test, y_test))
        accuracies = cross_val_score(estimator=reg, X=X, y=y, cv=10)
        print("Cross-validation-accuracies")
        print(accuracies)
        print("Cross-validation-accuracies Mean:")
        print(accuracies.mean())
        print("Cross-validation-accuracies Standard Devaition:")
        print(accuracies.std())
        pickle.dump(reg, open(filename, 'wb'))
    
    def linear_reg(self, X,y):
        np_x_train, np_x_test, np_y_train, np_y_test = train_test_split( X, y, shuffle=True, test_size=0.90, random_state=42)
        np_x_train=np.array(np_x_train)
        np_y_train=np.array(np_y_train)
        np_x_test=np.array(np_x_test)
        np_y_test=np.array(np_y_test)        
        Y_pred = np.empty((np.shape(np_y_test)[0], np.shape(np_y_test)[1]))
        bias = 1
        for i in range(9):
            y = np_y_train[:, i]
            W = np.linalg.inv(np_x_train.T @ np_x_train) @ np_x_train.T @ y
            W = [weight + bias for weight in W]
            y_pred = np_x_test @ W
            Y_pred[:, i] = y_pred

        Y_pred = (Y_pred == Y_pred.max(axis=1)[:, None]).astype(int)

        total_acc = np.empty(9)
        for i in range(9):
            total_acc[i] = self.get_accuracy_score(np_y_test[:, i],
                                                    Y_pred[:, i], normalized=False)

        acc = np.sum(total_acc) / (np.shape(np_y_test)[0] * 9)
        print("Accuracy LR: {0}".format(acc))

    def get_accuracy_score(self,y_true, y_pred, normalized=True):
        pred_accu = accuracy_score(y_true, y_pred, normalize=normalized)
        return pred_accu

if __name__ == '__main__':
    obj = Regressor()
    data_multiple = pd.read_csv('tictac_multi.txt', sep=" ", header=None)
    col_X = [0, 1, 2, 3, 4, 5, 6, 7, 8]
    col_y = [9, 10, 11, 12, 13, 14, 15, 16, 17]
    X_multiple = data_multiple[col_X]
    y_multiple=data_multiple[col_y]

    print("For intermediate boards optimal play(multi-label) dataset:")
    print("KNN Regressor:")
    for i in col_y:
        filename = 'Model_param_col_'+str(i-9)+'.pkl'
        print("For column no: "+str(i-9))
        reg_knn = KNeighborsRegressor(n_neighbors=9,weights='distance',algorithm='kd_tree',p=2,leaf_size=9,n_jobs=-1)
        obj.funcreg(reg_knn, X_multiple, data_multiple[i],filename)

    print("Linear Regressor:")
    obj.linear_reg(X_multiple,y_multiple)

    print("MLP Regressor:")
    for i in col_y:
        filename = 'Model_param_col_'+str(i-9)+'.pkl'
        print("For column no: "+str(i-9))
        reg_knn = MLPRegressor(random_state=42, max_iter=100,early_stopping=True,solver='lbfgs', activation='tanh')
        obj.funcreg(reg_knn, X_multiple, data_multiple[i],filename)


For intermediate boards optimal play(multi-label) dataset:
KNN Regressor:
For column no: 0
Confusion_matrix for initial training-test set
[[4333  157]
 [ 613  793]]
Classification_report for initial training-test set
              precision    recall  f1-score   support

           0       0.88      0.97      0.92      4490
           1       0.83      0.56      0.67      1406

    accuracy                           0.87      5896
   macro avg       0.86      0.76      0.80      5896
weighted avg       0.87      0.87      0.86      5896

Cross-validation-accuracies
[0.74449855 0.858954   0.83401277 0.8765574  0.87361868 0.92266674
 0.87459051 0.92029613 0.90131005 0.9193329 ]
Cross-validation-accuracies Mean:
0.872583772326226
Cross-validation-accuracies Standard Devaition:
0.05088777617738081
For column no: 1
Confusion_matrix for initial training-test set
[[4729  124]
 [ 676  367]]
Classification_report for initial training-test set
              precision    recall  f1-score   suppor

Confusion_matrix for initial training-test set
[[4123  388]
 [ 278 1107]]
Classification_report for initial training-test set
              precision    recall  f1-score   support

           0       0.94      0.91      0.93      4511
           1       0.74      0.80      0.77      1385

    accuracy                           0.89      5896
   macro avg       0.84      0.86      0.85      5896
weighted avg       0.89      0.89      0.89      5896

Cross-validation-accuracies
[0.59979641 0.60578915 0.64443244 0.63068992 0.64357378 0.64287306
 0.70780773 0.73317232 0.65944003 0.71857514]
Cross-validation-accuracies Mean:
0.6586149986612544
Cross-validation-accuracies Standard Devaition:
0.04390878809952901
For column no: 3
Confusion_matrix for initial training-test set
[[4490  346]
 [ 402  658]]
Classification_report for initial training-test set
              precision    recall  f1-score   support

           0       0.92      0.93      0.92      4836
           1       0.66      0.62